# Quickstart

# Install

First, install the following modules via pip—we require Python ≥ 3.9:

```
pip install rush-py
```

# Generate small molecule conformers

One of the simplest things you can do with Rush is generate 3D small molecule conformers from SMILES using the Auto3D module. We will break down how to do this step-by-step, but lets start with the full code:

In [ ]:
import rush

client = rush.build_blocking_provider_with_functions(access_token = "PUT_YOUR_TOKEN_HERE")

# setup an SMI file that contains the SMILES string of our ligand
ligand_smi_filename = client.workspace / "ligand.smi"
ligand_smi_filename.write_text(
    "c1nc(c2c(n1)n(cn2)[C@H]3[C@@H]([C@@H]([C@H](O3)CO[P@@](=O)(O)O[P@](=O)(O)OP(=O)(O)O)O)O)N 1"
)

# run Auto3D which will give us 3 conformers of our ligand
# in the SDF format and the QDXF format
ligand_sdf_obj, ligand_qdxf_obj = client.auto3d(
    ligand_smi_filename, # the filename that stores our ligand
    "smi",               # the format of the file
    { "k": 3 },          # the number of conformers to generate
    resources={
        "gpus": 1,            # the number of GPUs to use
        "storage": 5,         # the amount of storage to use
        "storage_units": "MB" # the units of storage (here we are using megabytes)
    },
)

# print the status of all jobs
print(client.status())

# download the results (this will block until the Auto3D job has completed)
ligand_sdf = ligand_sdf_obj.download()
ligand_qdxf = ligand_qdxf_obj.download()

print(ligand_sdf.read_text())  # print the SDF version of the result
print(ligand_qdxf.read_text()) # print the QDXF version of the result

# Step-by-step

## Import the Rush Python library

The first thing we do is import the `rush` Python library:

In [ ]:
import rush

## Create a Rush client

The next step is to create a client. This client will be used to interact with the Rush API. You will need to provide your Access Token to authenticate with the API. You can get your Access Token by signing up at [https://rush.qdx.co](https://rush.qdx.co) and going into your account settings.

Usually, you should store your Access Token somewhere safe and not hardcode it into your scripts (e.g. in a configuration file or environment variable). For the sake of this example, we will hardcode it:

In [ ]:
client = rush.build_blocking_provider_with_functions(access_token = "PASTE_YOUR_TOKEN_HERE")

But specifying that we want a "blocking provider" we are telling Rush that whenever we interact with the API we want to wait for the response before continuing. This is useful for simple scripts that are not running lots of jobs in parallel.

## Create an example SMILES file

To run Auto3D we need to specify the SMILES strings for which we want 3D conformers to be generated. These SMILES strings must be stored in a `.smi` file. For this example, we will use a sample file that contains the SMILES strings for one simple small molecule:

In [ ]:
# setup an SMI file that contains the SMILES string of our ligand
ligand_smi_filename = client.workspace / "ligand.smi"
ligand_smi_filename.write_text(
    "c1nc(c2c(n1)n(cn2)[C@H]3[C@@H]([C@@H]([C@H](O3)CO[P@@](=O)(O)O[P@](=O)(O)OP(=O)(O)O)O)O)N 1"
)

### A small note about workspaces

By default, the `client.workspace` will be your current working directory, so after this code runs you should see a file in your current working directory called `"ligand.smi"`. If you want to specify a different workspace directory, you can do so by specifying the `workspace` argument when building the client:

In [ ]:
# instead of creating a client with this code
client = rush.build_blocking_provider_with_functions(access_token = "PASTE_YOUR_TOKEN_HERE")

# you can create a client with this code, and explicitly set the workspace
client = rush.build_blocking_provider_with_functions(
    access_token = "PUT_YOUR_TOKEN_HERE",
    workspace="PUT_YOUR_PREFERRED_DIRECTORY_HERE"
)

##  Run Auto3D

Now that we have our SMILES file, we can use it as input to run the Auto3D module. This will generate 3D conformers (in a variety of possible protonation states) for each SMILES string in the SMILES file:

In [ ]:
# run Auto3D which will give us 3 conformers of our ligand
# in the SDF format and the QDXF format
ligand_sdf_obj, ligand_qdxf_obj = client.auto3d(
    ligand_smi_filename, # the filename that stores our ligand
    "smi",               # the format of the file
    { "k": 3 },          # the number of conformers to generate
    resources={
        "gpus": 1,            # the number of GPUs to use
        "storage": 5,         # the amount of storage to use
        "storage_units": "MB" # the units of storage (here we are using megabytes)
    },
)

### A small note about resources

In addition to their module-specific inputs, all modules also accept the `resource=` parameter. This parameter specifies the computational resources that you want to use to run the module. In this example, we have asked Rush to run the Auto3D module using 1 GPU and 5 megabytes of storage space.

## See the job status

Calling `client.auto3d` will tell Rush to run a new Auto3D job. However, it will not wait for the job to complete before continuing. This is convenient, because sometimes jobs can take a long time to run, and we might have other code we want to run in the meantime. If we want to track the status of all of our jobs, we can use the `client.status` function:

In [ ]:
# print the status of all jobs
print(client.status())

## Download the results

After calling `client.auto3d`, we got back two objects: `ligand_sdf_obj` and `ligand_qdxf_obj`. These objects are references to the results of the Auto3D job. They are stored in Rush and we can access them from anywhere at any time. We can use these objects as inputs to other modules, we can download them, and we can even use them to check the status of the Auto3D job that we ran.

For now, we simply download the results and print them out:

In [ ]:
# download the results (this will block until the Auto3D job has completed)
ligand_sdf = ligand_sdf_obj.download()
ligand_qdxf = ligand_qdxf_obj.download()

print(ligand_sdf.read_text())  # print the SDF version of the result
print(ligand_qdxf.read_text()) # print the QDXF version of the result

If you want to find the actual files, they will be in the `objects` directory inside your `client.workspace` directory. Remember, by default, the `client.workspace` is the current working directory.

### A note about objects

For most things that we are interested in doing, we do not have to wait for the job created by `client.auto3d` to actually complete. We can start using `ligand_sdf_obj` and `ligand_qdxf_obj` straight away. For example, we could pass them as inputs to a molecular dynamics simulation job. This would tell Rush to automatically run the molecular dynamics simulation job as soon as the Auto3D job completes.

However, the `download` function is kind of special and will explicitly block our program from continuing until the Auto3D job is complete. This is because `download` actually fetches the contents of the objects from Rush, and to do so it needs to be sure the contents actually exists.

# Next steps

In this quickstart, we generated 3D small molecule conformers from SMILES strings using the Auto3D module. We learned how to:

1. Create a client
2. Run the Auto3D module
3. Check the status of the job
4. Download the results

Checkout our other quickstarts to see how to use other modules. For example, now that we have some 3D small molecule conformers, we can run molecular dynamics simulation, quantum energy calculations, quantum geometry optimizations, docking, and much more!